<a href="https://colab.research.google.com/github/harperd17/energy_volatility_prediction/blob/main/notebooks/Modeling/Model_Result_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The goal of this notebook is to visualize experiments from Neptune

## This will set the prefix of your log file name

In [1]:
name = "Adam"
#Not grid "adamgoldsteindatascience/energyprice-volatility-grid"

neptune_project_name = "adamgoldsteindatascience/energyprice-volatility"

#Overall Reproduceability
random_state = 0

In [2]:
# Clones the main branch githubt to import: 
# 1.from fengine import Fengine
# 2.from sklearnlogger import SklearnLogger
!git clone https://github.com/harperd17/energy_volatility_prediction.git

Cloning into 'energy_volatility_prediction'...
remote: Enumerating objects: 681, done.
remote: Counting objects: 100% (681/681), done.
remote: Compressing objects: 100% (662/662), done.
remote: Total 681 (delta 417), reused 16 (delta 9), pack-reused 0
Receiving objects: 100% (681/681), 19.74 MiB | 5.44 MiB/s, done.
Resolving deltas: 100% (417/417), done.


In [3]:
!pip install neptune-client --quiet
import neptune.new as neptune

     |████████████████████████████████| 276 kB 9.0 MB/s 
     |████████████████████████████████| 829 kB 58.2 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 180 kB 20.1 MB/s 
     |████████████████████████████████| 131 kB 50.3 MB/s 
     |████████████████████████████████| 8.4 MB 44.4 MB/s 
     |████████████████████████████████| 79 kB 4.5 MB/s 
     |████████████████████████████████| 138 kB 51.1 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 127 kB 52.4 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 130 kB 50.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
import pandas as pd
import numpy as np
import sys
sys.path.append("energy_volatility_prediction/helpers/")

from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import cross_validate

import matplotlib.pyplot as plt

from fengine import Fengine
from sklearnlogger import SklearnLogger
import warnings

from google.colab import drive

drive.mount('./drive/')

%cd drive/Shareddrives/Data606_Energy

Mounted at ./drive/
/content/drive/Shareddrives/Data606_Energy


# Logger and Plotting Functions

In [5]:
import json
def get_neptune_key(username):

    with open("creds/creds.json","r") as f:

        api_token = json.load(f)["Users"][username]["neptune_key"]

    return api_token





# Analyze Results

In [6]:
# Fetch project 'jack/sandbox'
project = neptune.get_project(name=neptune_project_name,api_token=get_neptune_key(name))

# Fetch all Runs metadata as Pandas DataFrame
runs_table_df = project.fetch_runs_table().to_pandas()

runs_table_df.columns =  [col.split("/")[2] if "Parameters" in col else col.split("/")[1] for col in runs_table_df.columns]

In [7]:
runs_table_df.sort_values("Adj_R2_test",ascending=False)["Cust_Columns"][5]

"[['index', 'Year', 'State', 'Months Above Average Price Movement BZ=F', 'Months Above Average Price Volume BZ=F', 'Months Above Average Price Movement NG=F', 'Months Above Average Price Volume NG=F', 'Months Above Average Price Movement CL=F', 'Months Above Average Price Volume CL=F', 'Months Above Average Price Movement HO=F', 'Months Above Average Price Volume HO=F', 'Hot Summer Months', 'Cold Winter Months', 'Dry Summer Months', 'COV', 'target_COV', 'R_AB_MWh', 'R_AB_count', 'R_BFG_MWh', 'R_BFG_count', 'R_BIT_MWh', 'R_BIT_count', 'R_BLQ_MWh', 'R_BLQ_count', 'R_DFO_MWh', 'R_DFO_count', 'R_GEO_MWh', 'R_GEO_count', 'R_JF_MWh', 'R_JF_count', 'R_KER_MWh', 'R_KER_count', 'R_LFG_MWh', 'R_LFG_count', 'R_LIG_MWh', 'R_LIG_count', 'R_MSB_MWh', 'R_MSB_count', 'R_MSN_MWh', 'R_MSN_count', 'R_MSW_MWh', 'R_MSW_count', 'R_MWH_MWh', 'R_MWH_count', 'R_NG_MWh', 'R_NG_count', 'R_NUC_MWh', 'R_NUC_count', 'R_OBG_MWh', 'R_OBG_count', 'R_OBS_MWh', 'R_OBS_count', 'R_OG_MWh', 'R_OG_count', 'R_OTH_MWh', 'R_OT

In [8]:
runs_table_df['creation_time'] = pd.to_datetime(runs_table_df['creation_time'])

runs_table_df['Cust_Model'] = runs_table_df['Cust_Model'].apply(lambda model: "XGBRegressor" if model == "type" else model)

In [9]:
runs_table_df.query('Adj_R2_test > 0.40').sort_values("Adj_R2_test",ascending=False).head()

,creation_time,description,failed,hostname,id,modification_time,monitoring_time,name,owner,ping_time,running_time,size,state,tags,Adj_R2_test,Adj_R2_train,Coef_train,Cust_Columns,Cust_Model,Cust_Pipeline_funcs,Explained_Variance_Score_test,Explained_Variance_Score_train,MSE_test,MSE_train,ET,ET__bootstrap,ET__ccp_alpha,ET__criterion,ET__max_depth,ET__max_features,ET__max_leaf_nodes,ET__max_samples,ET__min_impurity_decrease,ET__min_samples_leaf,ET__min_samples_split,ET__min_weight_fraction_leaf,ET__n_estimators,ET__n_jobs,ET__oob_score,ET__random_state,...,missing,monotonic_cst,n_estimators,n_iter,n_iter_no_change,n_jobs,n_nonzero_coefs,n_subsamples,normalize,nthread,objective,oob_score,positive,precompute,random_state,reg_alpha,reg_lambda,scale_pos_weight,scoring,seed,selection,silent,solver,subsample,threshold_lambda,tol,validation_fraction,verbose,verbosity,warm_start,weights,R2_test,R2_train,y_preds_test,y_preds_train,cpu,gpu_memory,memory,stderr,stdout
71,2021-12-01 22:20:08.350000+00:00,,False,94a359147139,EN-1683,2021-12-01 22:20:09.360000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-01 22:20:09.360000+00:00,1.010,16892.0,idle,,0.769082,0.812980,None,"[['index', 'Year', 'State', 'Months Above Aver...",AdaBoostRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000176,0.000141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.774601,0.817450,[0.01151657 0.02170265 0.0462405 0.02251547 0...,[0.03318682 0.04184224 0.05325788 0.03969497 0...,54.1,NaN,1.222168,NaN,"b'Shutting down background jobs, please wait a..."
52,2021-12-01 23:03:53.033000+00:00,,False,94a359147139,EN-1702,2021-12-01 23:03:54.023000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-01 23:03:54.023000+00:00,0.989,16703.0,idle,,0.768890,0.769240,[0.41791302 0.08612174 0.01242776 0.02825161 0...,"[['index', 'Year', 'State', 'Months Above Aver...",XGBRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000176,0.000174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,NaN,500.0,NaN,NaN,1,NaN,NaN,NaN,None,reg:linear,NaN,NaN,NaN,0,0.0,1.0,1.0,NaN,None,NaN,None,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.774413,0.774755,[0.01164189 0.02066103 0.04423982 0.02363828 0...,[0.03153768 0.04303721 0.05009851 0.04252222 0...,1.5,NaN,1.221241,NaN,"b'Shutting down background jobs, please wait a..."
54,2021-12-01 22:41:40.726000+00:00,,False,94a359147139,EN-1700,2021-12-01 22:41:41.719000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-01 22:41:41.719000+00:00,0.992,16304.0,idle,,0.767606,0.812027,None,"[['index', 'Year', 'State', 'Months Above Aver...",HistGradientBoostingRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000177,0.000142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,loss,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e-07,0.1,0,NaN,False,NaN,0.773160,0.816519,[0.0101247 0.02004124 0.0464448 0.02185191 0...,[0.03524172 0.041013 0.05224845 0.04109402 0...,2.3,NaN,1.221451,NaN,"b'Shutting down background jobs, please wait a..."
60,2021-12-01 22:40:02.029000+00:00,,False,94a359147139,EN-1694,2021-12-01 22:40:02.945000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-01 22:40:02.945000+00:00,0.915,16313.0,idle,,0.767604,0.812035,None,"[['index', 'Year', 'State', 'Months Above Aver...",HistGradientBoostingRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000177,0.000142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,loss,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e-07,0.1,0,NaN,False,NaN,0.773158,0.816527,[0.01012712 0.0200414 0.04644496 0.02185139 0...,[0.03524224 0.04101182 0.0522482 0.04108958 0...,8.5,NaN,1.220982,NaN,"b'Shutting down back

In [10]:


runs_table_df["Adj_R2_test_train_diff"] = runs_table_df["Adj_R2_test"] - runs_table_df["Adj_R2_train"]

In [11]:
runs_table_df

,creation_time,description,failed,hostname,id,modification_time,monitoring_time,name,owner,ping_time,running_time,size,state,tags,Adj_R2_test,Adj_R2_train,Coef_train,Cust_Columns,Cust_Model,Cust_Pipeline_funcs,Explained_Variance_Score_test,Explained_Variance_Score_train,MSE_test,MSE_train,ET,ET__bootstrap,ET__ccp_alpha,ET__criterion,ET__max_depth,ET__max_features,ET__max_leaf_nodes,ET__max_samples,ET__min_impurity_decrease,ET__min_samples_leaf,ET__min_samples_split,ET__min_weight_fraction_leaf,ET__n_estimators,ET__n_jobs,ET__oob_score,ET__random_state,...,monotonic_cst,n_estimators,n_iter,n_iter_no_change,n_jobs,n_nonzero_coefs,n_subsamples,normalize,nthread,objective,oob_score,positive,precompute,random_state,reg_alpha,reg_lambda,scale_pos_weight,scoring,seed,selection,silent,solver,subsample,threshold_lambda,tol,validation_fraction,verbose,verbosity,warm_start,weights,R2_test,R2_train,y_preds_test,y_preds_train,cpu,gpu_memory,memory,stderr,stdout,Adj_R2_test_train_diff
0,2021-12-04 01:38:11.537000+00:00,,False,bd3aa411b1f1,EN-1754,2021-12-04 01:38:12.664000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-04 01:38:12.664000+00:00,1.127,17352.0,idle,,0.605281,0.721970,None,"[['index', 'Year', 'State', 'Months Above Aver...",LinearRegression,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000266,0.000216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,deprecated,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.613388,0.727681,[0.02202831 0.06679748 0.03679194 0.05828421 0...,[0.06716892 0.05080912 0.03338024 0.06582761 0...,19.2,NaN,0.965393,NaN,"b'Shutting down background jobs, please wait a...",-0.116689
1,2021-12-04 01:38:09.912000+00:00,,False,bd3aa411b1f1,EN-1753,2021-12-04 01:38:11.072000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-04 01:38:11.072000+00:00,1.159,18095.0,idle,,0.646886,0.841413,[0.26673083 0.63214091 0.01637803 0.0148986 0...,"[['index', 'Year', 'State', 'Months Above Aver...",RandomForestRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000238,0.000123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,50.0,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,False,NaN,0.654138,0.844670,[0.02939691 0.06232484 0.03045787 0.0477998 0...,[0.07217043 0.06005891 0.03330494 0.05679886 0...,15.4,NaN,0.965794,NaN,"b'Shutting down background jobs, please wait a...",-0.194527
2,2021-12-04 01:38:08.059000+00:00,,False,bd3aa411b1f1,EN-1752,2021-12-04 01:38:09.289000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-04 01:38:09.289000+00:00,1.229,18258.0,idle,,0.666835,0.946903,[0.15601041 0.66843795 0.02450343 0.01460787 0...,"[['index', 'Year', 'State', 'Months Above Aver...",GradientBoostingRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000224,0.000041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,50.0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.000000e-04,0.1,0,NaN,False,NaN,0.673678,0.947994,[0.03672114 0.06716081 0.02199665 0.05387986 0...,[0.0839029 0.0698209 0.03679187 0.0598314 0...,16.0,NaN,0.965588,NaN,"b'Shutting down background jobs, please wait a...",-0.280069
3,2021-12-04 01:38:06.331000+00:00,,False,bd3aa411b1f1,EN-1751,2021-12-04 01:38:07.464000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-04 01:38:07.464000+00:00,1.132,17965.0,idle,,0.578379,0.662556,None,"[['index', 'Year', 'State', 'Months Above Aver...",HistGradientBoostingRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000284,0.000263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,loss,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e-07,0.1,0,NaN,False,NaN,0.587038,0.669487,[0.03075631 0.05523061 0.04288268 0.04759007 0...,[0.0541728

In [12]:

df_max = runs_table_df[["Cust_Model","base_estimator","Adj_R2_test","Adj_R2_train","MSE_test","Adj_R2_test_train_diff","n_estimators","learning_rate","max_depth","Cust_Columns","Cust_Pipeline_funcs","Coef_train"]].reset_index()

In [13]:
#Unique Experiments

df_max["Cust_Columns"].unique().shape

(15,)

In [14]:
vis = df_max.query('Adj_R2_test >= 0.60').query('Adj_R2_test_train_diff < 0.03').query('Adj_R2_train < 0.79').drop(columns="Adj_R2_test_train_diff").sort_values("Adj_R2_test",ascending=False).drop_duplicates(["Cust_Model","n_estimators","learning_rate","max_depth"])

In [15]:
def highlight_max(x, color):
    return np.where(x == np.nanmax(x.to_numpy()), f"background-color:{color};", None)

In [21]:
model_summary = vis[["Cust_Model","base_estimator","Adj_R2_test"]].groupby("Cust_Model").max().sort_values("Adj_R2_test",ascending=False)


model_summary["base_estimator"] = model_summary["base_estimator"].apply(lambda x: x if x == np.nan else str(x).split("(")[0])

model_summary.style.apply(highlight_max, color='lightgreen',subset=["Adj_R2_test"])



,base_estimator,Adj_R2_test
Cust_Model,,
XGBRegressor,nan,0.768890
HistGradientBoostingRegressor,nan,0.766902
BaggingRegressor,HistGradientBoostingRegressor,0.763546
AdaBoostRegressor,LassoLarsIC,0.763318
ExtraTreesRegressor,nan,0.713314
GradientBoostingRegressor,nan,0.695125
RandomForestRegressor,nan,0.685285
VotingRegressor,nan,0.674194
BayesianRidge,nan,0.639789


In [22]:
a = vis[["Cust_Model","base_estimator","Adj_R2_test","Cust_Columns","Coef_train"]].groupby("Cust_Model").max().sort_values("Adj_R2_test",ascending=False)

a["col_len"] = a["Cust_Columns"].apply(lambda x: len(x[0]))
a["coef_len"] = a["Coef_train"].apply(lambda x: len(x))

In [23]:
#Unique Experiments

unique_experiments = df_max.drop_duplicates(["Cust_Model","n_estimators","learning_rate","max_depth"]).reset_index()

unique_experiments

,level_0,index,Cust_Model,base_estimator,Adj_R2_test,Adj_R2_train,MSE_test,Adj_R2_test_train_diff,n_estimators,learning_rate,max_depth,Cust_Columns,Cust_Pipeline_funcs,Coef_train
0,0,0,LinearRegression,NaN,0.605281,0.721970,0.000266,-0.116689,NaN,NaN,NaN,"[['index', 'Year', 'State', 'Months Above Aver...","[[['scale_month_columns', None], ['drop_column...",None
1,1,1,RandomForestRegressor,NaN,0.646886,0.841413,0.000238,-0.194527,50.0,NaN,6,"[['index', 'Year', 'State', 'Months Above Aver...","[[['scale_month_columns', None], ['drop_column...",[0.26673083 0.63214091 0.01637803 0.0148986 0...
2,2,2,GradientBoostingRegressor,NaN,0.666835,0.946903,0.000224,-0.280069,50.0,0.100000,6,"[['index', 'Year', 'State', 'Months Above Aver...","[[['scale_month_columns', None], ['drop_column...",[0.15601041 0.66843795 0.02450343 0.01460787 0...
3,3,3,HistGradientBoostingRegressor,NaN,0.578379,0.662556,0.000284,-0.084177,NaN,0.050000,4,"[['index', 'Year', 'State', 'Months Above Aver...","[[['scale_month_columns', None], ['drop_column...",None
4,4,4,BayesianRidge,NaN,0.612736,0.721611,0.000261,-0.108875,NaN,NaN,NaN,"[['index', 'Year', 'State', 'Months Above Aver...","[[['scale_month_columns', None], ['drop_column...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,751,751,GradientBoostingRegressor,NaN,0.281383,1.000000,0.000212,-0.718617,1635.0,0.007611,10,"[['index', 'COV', 'COM_percapita_mean_kWh_elec...","[[['average_weather_columns', 'Temp Standard D...",None
226,752,752,GradientBoostingRegressor,NaN,0.322447,1.000000,0.000202,-0.677553,942.0,0.017769,16,"[['index', 'COV', 'COM_percapita_mean_kWh_elec...","[[['average_weather_columns', 'Temp Standard D...",None
227,753,753,HistGradientBoostingRegressor,NaN,0.219787,0.977071,0.000230,-0.757284,NaN,0.119865,15.5505,"[['index', 'COV', 'COM_percapita_mean_kWh_elec...","[[['average_weather_columns', 'Temp Standard D...",None
228,754,754,HistGradientBoostingRegressor,NaN,0.178723,0.942678,0.000241,-0.763954,NaN,0.127300,27.0675,"[['index', 'COV', 'COM_percapita_mean_kWh_elec...","[[['average_weather_columns', 'Temp Standard D...",None


In [24]:
# df=df_max.sort_values(by=['Adj_R2_test', 'Adj_R2_train'],inplace=True, ascending=False)
# #df.sort_values(by=['Brand'], inplace=True, ascending=False)


In [31]:
runs_table_df["creation_time"] = list(runs_table_df.index)



fig = (px.line(runs_table_df.query('Adj_R2_test > 0.40').sort_values("creation_time",ascending=False),x="creation_time",y="Adj_R2_test",color="Cust_Model",hover_data=["Adj_R2_test","Adj_R2_test"],
        
        log_y=True,title="Model Performance - Adj R2")
        
        .for_each_trace(lambda t: t.update(name=t.name.replace("Cust_Model=","")))
        
        )


fig.update_layout(
    xaxis_title="Index",)

In [30]:

(px.line(runs_table_df.query('Adj_R2_test > 0.40').sort_values("creation_time",ascending=False),x="creation_time",y="MSE_test",color="Cust_Model",hover_data=["Adj_R2_test","Adj_R2_test"],
        
        log_y=True,title="Model Performance - MSE Test")
        
        .for_each_trace(lambda t: t.update(name=t.name.replace("Cust_Model=","")))
        
        )